# Day 24 
## Part 1

We want
$$
p + av = q + bu
$$
for positive $a$ and $b$ within the bounds set.
So solve the simultaneous equations 
$$
p_x + av_x = q_x + bu_x
$$
$$
p_y + av_y = q_y + bu_y
$$
so
$$
b = \frac{p_x + av_x - q_x}{u_x}
$$
$$
v_ya = q_y + (p_x + av_x - q_x)\frac{u_y}{u_x} - p_y  
$$
$$
(v_y - \frac{v_xu_y}{u_x})a = q_y + {p_x - q_x}\frac{u_y}{u_x} - p_y
$$
$$
a = \frac{q_yu_x + p_xu_y - q_xu_y - p_yu_x}{v_yu_x - v_xu_y}
$$
and substitute for b.

In [1]:
from dataclasses import dataclass

@dataclass
class Point3D:
    x: int
    y: int
    z: int

    def __add__(self, other):
        return self.__class__(self.x + other.x, self.y + other.y, self.z + other.z)

    def __sub__(self, other):
        return self.__class__(self.x - other.x, self.y - other.y, self.z - other.z)

    def __neg__(self):
        return self.__class__(-self.x, -self.y, -self.z)

    def __hash__(self):
        return hash((self.x, self.y, self.z))

    def __iter__(self):
        yield self.x
        yield self.y
        yield self.y

    def __mod__(self, other):
        if isinstance(other, Point):
            return self.__class__(self.x % other.x, self.y % other.y, self.z % other.z)
        else:
            return self.__class__(self.x % other, self.y % other, self.z % other)
        
    def __mul__(self, multiple):
        return self.__class__(self.x * multiple, self.y * multiple, self.z * multiple)

def intersection_times(p, v, q, u):
    a = (q.y * u.x + p.x * u.y - q.x * u.y - p.y * u.x) / (v.y * u.x - v.x * u.y)
    b = (p.x + a * v.x - q.x) / u.x
    return (a, b)

def parse_data(s):
    data = []
    for line in s.strip().splitlines():
        p, v = [Point3D(*map(int, xs.split(", "))) for xs in line.split(" @ ")]
        data.append((p, v))
    return data

test_data = parse_data("""19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3""")

test_data

[(Point3D(x=19, y=13, z=30), Point3D(x=-2, y=1, z=-2)),
 (Point3D(x=18, y=19, z=22), Point3D(x=-1, y=-1, z=-2)),
 (Point3D(x=20, y=25, z=34), Point3D(x=-2, y=-2, z=-4)),
 (Point3D(x=12, y=31, z=28), Point3D(x=-1, y=-2, z=-1)),
 (Point3D(x=20, y=19, z=15), Point3D(x=1, y=-5, z=-3))]

In [2]:
p, v = test_data[0]
q, u = test_data[1]
a, b = intersection_times(p, v, q, u)
print(a, b)
p + v * a

2.3333333333333335 3.666666666666668


Point3D(x=14.333333333333332, y=15.333333333333334, z=25.333333333333332)

In [3]:
from itertools import combinations, chain

def intersections(data, lbound, ubound):
    total = 0
    for (p, v), (q, u) in combinations(data, 2):
        try:
            a, b = intersection_times(p, v, q, u)
            if a > 0 and b > 0:
                newp = p + v * a
                newq = q + u * b
                if all(lbound <= n <= ubound for n in (newp.x, newp.y, newq.x, newq.y)):
                    total += 1
        # parallel vectors
        except ZeroDivisionError:
            pass
    return total

intersections(test_data, 7, 27)        

2

In [4]:
data = parse_data(open("input").read())

intersections(data, 200000000000000, 400000000000000)

11995

## Part 2
Matching one hailstone means the throw could be from anywhere, matching two defines a plane, matching three gives the throw location on the plane. Presumably the problem is constructed so the rest of the hailstones are hit by the defined throw. I made a mistake first time round even in the algebra above so I think I'll get some help with this one.

In [5]:
data[:3]

[(Point3D(x=288998070705911, y=281498310692304, z=225433163951734),
  Point3D(x=-63, y=25, z=66)),
 (Point3D(x=267942038821112, y=319206560980050, z=228821793591214),
  Point3D(x=97, y=-61, z=70)),
 (Point3D(x=444805588706877, y=248504563833176, z=237588696365934),
  Point3D(x=-167, y=337, z=94))]

In [6]:
import sympy

def intersection(data):
    x, y, z, vx, vy, vz, *ts = sympy.symbols("x y z vx vy vz t0 t1 t2")
    return sympy.solve(
        [
            x + ts[i] * vx - q.x - ts[i] * u.x for i, (q, u) in enumerate(data[:3])
        ] +
        [
            y + ts[i] * vy - q.y - ts[i] * u.y for i, (q, u) in enumerate(data[:3])
        ] +
        [
            z + ts[i] * vz - q.z - ts[i] * u.z for i, (q, u) in enumerate(data[:3])
        ],
        [x, y, z, vx, vy, vz, *ts]
    )

intersection(test_data)

[(24, 13, 10, -3, 1, 2, 5, 3, 4)]

In [7]:
x, y, z, *rest = intersection(data)[0]

In [8]:
int(x + y + z)

983620716335751